In [1]:
import pandas as pd
from models.lightfm import SimpleLightFM, FeaturedLightFM, WeightFeaturedLightFM
from models.hybrid import TwoStepRecommender, CombineRecommender, CombineUnseenRecommender
from models.popular import PopularUnseenRecommmender, PopularRecommender, SegmentUnseenRecommender

/Users/d-emeljanov/pet_projects/recsys-course/notebooks/venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.DataFrame([
    ('u1', 'i1', '2021-01-01', 90),
    ('u1', 'i2', '2021-01-01', 90),
    ('u2', 'i2', '2021-01-01', 90),
    ('u2', 'i3', '2021-01-01', 90),
    ('u3', 'i2', '2021-01-01', 90),
    ('u3', 'i3', '2021-01-01', 90),
    ('u4', 'i1', '2021-01-01', 90),
    ('u5', 'i2', '2021-01-01', 90),
    ('u6', 'i3', '2021-01-01', 90),
], columns=['uid', 'iid', 'dd', 'watched_pct'])

data['dd'] = pd.to_datetime(data['dd'])

if_df = pd.DataFrame([
    ('i1', 'f11', 'f21'),
    ('i2', 'f12', 'f22'),
    ('i3', 'f11', 'f23')
], columns=['iid', 'if1', 'if2'])

uf_df = pd.DataFrame([
    ('u1', 'age10'),
    ('u2', 'age10'),
    ('u3', 'age10'),
    ('u4', 'age10'),
    ('u5', 'age10'),
    ('u6', 'age10'),
], columns=['uid', 'uf1'])

unused_df = pd.DataFrame([
    ('u1', 'i3'),
    ('u2', 'i1'),
    ('u3', 'i1'),
    ('u4', 'i2'),
    ('u4', 'i3'),
    ('u5', 'i1'),
    ('u5', 'i3'),
    ('u6', 'i1'),
    ('u6', 'i2')
], columns=['uid', 'iid'])

In [4]:
model = CombineRecommender(
    models=['popular.PopularUnseenRecommmender'],
    models_n=[100],
    params=[
        '--days', '10'
    ],
    user_col='uid',
    item_col='iid',
    date_col='dd',
)

model.fit(data)

model.recommend(user_ids=['u4', 'u5', 'u6', 'u7'], N=10)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1359.69it/s]


0        [i2, i3]
1        [i1, i3]
2        [i1, i2]
3    [i1, i2, i3]
dtype: object

In [ ]:
model = PopularUnseenRecommmender(
    days=10,
    user_col='uid',
    item_col='iid',
    date_col='dd',
)

model.add_item_features(if_df)
model.add_user_features(uf_df)

model.fit(data)

model.recommend(user_ids=['u4', 'u5', 'u6', 'u7'], N=10)

In [ ]:
list(set(['if5', 'if2']))

In [ ]:
model = WeightFeaturedLightFM(
    no_components=10,
    user_col='uid',
    item_col='iid',
    date_col='dd',
    user_features_col=['uf1'],
    item_features_col=['if1'],
    preprocess_array_split=['if1']
)

# model = TwoStepRecommender(
#     models=['lightfm.FeaturedLightFM'],
#     models_n=[100],
#     final_model='xgboost.XGBoostRecommender',
#     params=[
#         '--days', '10',
#         '--watched_pct_min', '0',
#         '--user_features_col', 'uf1',
#         '--item_features_col', 'if1', 'if2',
#         '--category_features', 'uf1', 'if1', 'if2',
#     ],
#     user_col='uid',
#     item_col='iid',
#     date_col='dd'
# )

model.add_item_features(if_df)
model.add_user_features(uf_df)
model.add_unused(unused_df)

In [ ]:
model.fit(data)

In [ ]:
model.recommend(user_ids=['u4', 'u5', 'u6'], N=10)

In [ ]:
model.data.mapping()

In [ ]:
model.user_seen

In [ ]:
'u7' in data.groupby('uid')['iid'].apply(list).index.intersection(['u6', 'u7'])

In [ ]:
data['iid'].value_counts()

In [ ]:
data.groupby('uid')['iid'].value_counts().loc['u1'].head(1).index.tolist()